<a href="https://colab.research.google.com/github/James-H05/SSI_2021/blob/main/ee_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# import earth engine library - should be installed already in colab
import ee

In [3]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=O_kC6AwgHqOKaYYXBtwIyr2WjBQF0fsWmwIwfO3E3pg&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWhcBxVcSk1AHtNI14fA-bVGmG_b64U-Y5uOb1OCwi1kVaobTBQtLVI

Successfully saved authorization token.


In [21]:
terraclimate = ee.ImageCollection("IDAHO_EPSCOR/TERRACLIMATE").filter(ee.Filter.date('2017-07-01', '2017-08-01')).first()

In [22]:
france_geo = ee.Feature(ee.FeatureCollection('FAO/GAUL/2015/level0')
  .filter(ee.Filter.eq('ADM0_NAME', 'France'))
  .first())

In [36]:
# Reduce the region. The region parameter is the Feature geometry.
meanDictionary = terraclimate.reduceRegion(
  ee.Reducer.mean(),
  geometry=france_geo.geometry(),
  scale=30,
  maxPixels=100000000,
  bestEffort=True
)

# The result is a Dictionary.  Print it.
print(meanDictionary)

ee.Dictionary({
  "functionInvocationValue": {
    "functionName": "Image.reduceRegion",
    "arguments": {
      "bestEffort": {
        "constantValue": true
      },
      "geometry": {
        "functionInvocationValue": {
          "functionName": "Feature.geometry",
          "arguments": {
            "feature": {
              "functionInvocationValue": {
                "functionName": "Collection.first",
                "arguments": {
                  "collection": {
                    "functionInvocationValue": {
                      "functionName": "Collection.filter",
                      "arguments": {
                        "collection": {
                          "functionInvocationValue": {
                            "functionName": "Collection.loadTable",
                            "arguments": {
                              "tableId": {
                                "constantValue": "FAO/GAUL/2015/level0"
                              }
                    